In [1]:
from general_utils.atomic_facts import AtomicFactPromptGenerator
from openai import OpenAI
from general_utils.reader import JSONLineReader
from config import OPEN_AI_TOKEN

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/lukasellinger/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
client = OpenAI(api_key=OPEN_AI_TOKEN)
fh = JSONLineReader()
prompt_generator = AtomicFactPromptGenerator(demon_dir='../factscore/demos')

In [3]:
from datasets import load_dataset

dataset = load_dataset("lukasellinger/german_dpr_claim_verification_dissim-v1").get('train')

In [4]:
def get_facts(messages, model="gpt-3.5-turbo"):
    response = client.chat.completions.create(
    model=model,
    temperature=0.1,
    logprobs=True,
    top_logprobs=5,
    messages=messages,
    )
    txt_output = response.choices[0].message.content
    return txt_output

In [6]:
# Testing on a few examples
for entry in dataset.select(range(150, 160)):
    word = entry.get('english_word')
    claim = f"{word}: {entry['english_claim']}"
    
    messages = prompt_generator.create_prompt_from_sentence(claim)
    output = get_facts(messages)
    print(f"WORD: {word}\nOVERVIEW: {claim}\n\nOUTPUT: {output}")
    print("\n\n----------------------------\n\n")

WORD: YouTube channel
OVERVIEW: YouTube channel: defined as an airport from which one or more airlines offer an integrated service network to a variety of different destinations with a high frequency of connections

OUTPUT: 1. YouTube channel is defined as an airport.
2. YouTube channel is where one or more airlines offer an integrated service network.
3. YouTube channel offers services to a variety of different destinations.
4. YouTube channel has a high frequency of connections.


----------------------------


WORD: YouTube channel
OVERVIEW: YouTube channel: a synapse that becomes stronger by simultaneous activity in the pre- and post-synaptic neurons

OUTPUT: 1. YouTube channel is a synapse.
2. Simultaneous activity occurs in the pre- and post-synaptic neurons. 
3. The synapse becomes stronger.


----------------------------


WORD: Laservisiondisc
OVERVIEW: Laservisiondisc: the multiple excretion of ''too liquid'' chair

OUTPUT: 1. Laservisiondisc is involved.
2. The multiple excr

In [8]:
output

'1. Energy consumption is related to computer programs.\n2. Computer programs manage and store their individually required data using a database system.'

In [10]:
import re

for line in output.split('\n'):
    fact_match = re.match(r'^\d+\.(.*)', line)
    if fact_match:
        fact = fact_match.group(1).strip()
        print(fact)

Energy consumption is related to computer programs.
Computer programs manage and store their individually required data using a database system.
